## 20뉴스그룹 고급분류

In [2]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
news_data = fetch_20newsgroups(subset='all', random_state=156)

In [4]:
train_news = fetch_20newsgroups(subset='train', random_state=156, remove=('headers', 'footers', 'quotes'))

In [5]:
df_train =pd.DataFrame(train_news.data, columns=['data'])
df_train['target'] = train_news.target
df_train.isna().sum() #트레인에서 널값난값null nan확인. 0없다

data      0
target    0
dtype: int64

In [6]:
df_train[df_train.data == ''].count() 
#값이없는놈들이 218개나있네....훈련할방법이없음

data      218
target    218
dtype: int64

In [7]:
#빈것들은 과감히삭제 (행삭제)
df_train = df_train.drop(df_train[df_train.data == ''].index)
df_train[df_train.data == ''].count()

data      0
target    0
dtype: int64

In [8]:
#테스트데이터만들기
test_news = fetch_20newsgroups(subset='test', random_state=156, remove=('headers', 'footers', 'quotes'))

In [9]:
df_test =pd.DataFrame(test_news.data, columns=['data'])
df_test['target'] = test_news.target
df_test.isna().sum()  #테스트에도 널값난값null nan확인. 0없다

data      0
target    0
dtype: int64

In [10]:
df_test[df_test.data == ''].count() 
#값이없는놈들이 162개나있네....

data      162
target    162
dtype: int64

In [11]:
#빈것들은 과감히삭제 (행삭제)
df_test = df_test.drop(df_test[df_test.data == ''].index)
df_test[df_test.data == ''].count()

data      0
target    0
dtype: int64

In [12]:
X_train = df_train.data.values
y_train = df_train.target.values
X_test = df_test.data.values
y_test = df_test.target.values

In [14]:
X_train.shape, X_test.shape #(11096,)이것이 스트링의 어레이다 str의 array

((11096,), (7370,))

In [13]:
len(X_train), len(X_test) 
#기존 (11314, 7532)에서 줄었다  (11096, 7370) 

(11096, 7370)

In [16]:
#fetch로부터 전처리한(빈값삭제한것) 테스트데이터 저장(웹에서쓸꺼니까)
#트레인데이터는 모델만들면 안씀
df_test.to_csv('../static/data/news/test.csv', index=False)

## 피처 벡터화변환
- Case 1. CountVectorizer

In [17]:
# 트레인데이터를 숫자로 바꾸기 CountVectorizer 또는 TfidfVectorizer 사용해서
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() 
count_vect.fit(X_train)
X_train_count = count_vect.transform(X_train) 
X_test_count = count_vect.transform(X_test)
# 꼭 두단계를 거쳐야 train, test같은 인덱스를 가짐

In [18]:
X_train_count.shape, X_test_count.shape
#((11096, 101631), (7370, 101631)) 2차원
#문장 11096라인의 컬럼101631개

((11096, 101631), (7370, 101631))

In [21]:
count_vect.get_params()
#이정도수정할예정
#'max_df': 1.0,
#'stop_words': None,
#'ngram_range': (1, 1),

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.int64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'preprocessor': None,
 'stop_words': None,
 'strip_accents': None,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'vocabulary': None}

- Case 2. TfidVectorizer

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer()
tfidf_vect.fit(X_train)
X_train_tfidf = tfidf_vect.transform(X_train) 
X_test_tfidf = tfidf_vect.transform(X_test)
# 꼭 두단계를 거쳐야 train, test같은 인덱스를 가짐

In [20]:
X_train_tfidf.shape, X_test_tfidf.shape

((11096, 101631), (7370, 101631))

In [22]:
tfidf_vect.get_params()
#이정도수정할예정(똑같..)
#'max_df': 1.0,
#'stop_words': None,
#'ngram_range': (1, 1),

{'analyzer': 'word',
 'binary': False,
 'decode_error': 'strict',
 'dtype': numpy.float64,
 'encoding': 'utf-8',
 'input': 'content',
 'lowercase': True,
 'max_df': 1.0,
 'max_features': None,
 'min_df': 1,
 'ngram_range': (1, 1),
 'norm': 'l2',
 'preprocessor': None,
 'smooth_idf': True,
 'stop_words': None,
 'strip_accents': None,
 'sublinear_tf': False,
 'token_pattern': '(?u)\\b\\w\\w+\\b',
 'tokenizer': None,
 'use_idf': True,
 'vocabulary': None}

### Pipeline(모델결합)과 GridSearchCV를 통한 하이퍼 파라미터 튜닝
- Case A. CountVectorizer + LogisticRegression Classifier

In [23]:
#시간45분;;걸림...... 
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression

pipeline = Pipeline([ #리스트로주고, 안에 튜플로 씁니당
    ('count_vect', CountVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

In [24]:
params = {
    'count_vect__ngram_range': [(1,1), (1,2)], 
    'count_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
}

In [25]:
#여기서 진짜 45분걸린다아.......
from sklearn.model_selection import GridSearchCV

grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)
#18:52- 2*2*2*3=24fit 교수님34분소요
#나는 55분소요 300 (1,2) 1 0.7060

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 55.4min finished
{'count_vect__max_df': 300, 'count_vect__ngram_range': (1, 2), 'lr_clf__C': 1} 0.7060207025388442


In [26]:
best_count_lr = grid_pipe.best_estimator_
pred_count_lr = best_count_lr.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, pred_count_lr)

0.662415196743555

In [28]:
import joblib
joblib.dump(best_count_lr, '../static/model/news_count_lr.pkl')

['../static/model/news_count_lr.pkl']

- Case B. TfidfVectorizer + LogisticRegression Classifier

In [29]:
pipeline = Pipeline([ #리스트로주고, 안에 튜플로 씁니당
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('lr_clf', LogisticRegression())
])

params = {
    'tfidf_vect__ngram_range': [(1,1), (1,2)], 
    'tfidf_vect__max_df': [300, 700],
    'lr_clf__C': [1, 10]
}

In [30]:
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)
#교수님34분소요 c:10 700 (1,2) 0.7661
#19:56~ 1시간소요 10 700 (1,2) 0.7661

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  24 out of  24 | elapsed: 47.0min finished
{'lr_clf__C': 10, 'tfidf_vect__max_df': 700, 'tfidf_vect__ngram_range': (1, 2)} 0.766132057480442


In [31]:
best_tfidf_lr = grid_pipe.best_estimator_
pred_tfidf_lr = best_tfidf_lr.predict(X_test)
accuracy_score(y_test, pred_count_lr)
#교수님결과 0.7150
#결과값: 0.6624

0.662415196743555

In [32]:
joblib.dump(best_tfidf_lr, '../static/model/news_tfidf_lr.pkl')

['../static/model/news_tfidf_lr.pkl']

- Case C. TfidfVectorizer + Support Vector Classifier

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
pipeline = Pipeline([ #리스트로주고, 안에 튜플로 씁니당
    ('tfidf_vect', TfidfVectorizer(stop_words='english')),
    ('sv_clf', SVC())
])

params = {
    'tfidf_vect__ngram_range': [(1,1)],  #, (1,2)
    'tfidf_vect__max_df': [300], #, 700
    'sv_clf__C': [1] #, 10
}

In [5]:
from sklearn.model_selection import GridSearchCV
grid_pipe = GridSearchCV(pipeline, param_grid=params, cv=3, scoring='accuracy', verbose=1)
grid_pipe.fit(X_train, y_train)
print(grid_pipe.best_params_, grid_pipe.best_score_)
#교수님결과15분소요 10 300 (1,1) 0.7586
#파라미터줄여서 20:51~ 2분종료 ㅋ 0.7494

NameError: name 'X_train' is not defined

In [36]:
best_tfidf_sv = grid_pipe.best_estimator_
pred_tfidf_sv = best_tfidf_sv.predict(X_test)
accuracy_score(y_test, pred_tfidf_sv)
#교수님결과 0.6957
#결과값: 0.7035

0.7035278154681139

In [37]:
joblib.dump(best_tfidf_sv, '../static/model/news_tfidf_sv.pkl')

['../static/model/news_tfidf_sv.pkl']

## 모델저장 (불안해서 위에서 각각저장했음 ㅋ)

In [1]:
import joblib
joblib.dump(best_count_lr, '../static/model/news_count_lr.pkl')
joblib.dump(best_tfidf_lr, '../static/model/news_tfidf_lr.pkl')
joblib.dump(best_tfidf_sv, '../static/model/news_tfidf_sv.pkl')

NameError: name 'best_count_lr' is not defined

# TEST

In [2]:
import joblib
best_count_lr = joblib.load('../static/model/news_count_lr.pkl')
best_tfidf_lr = joblib.load('../static/model/news_tfidf_lr.pkl')
best_tfidf_sv = joblib.load('../static/model/news_tfidf_sv.pkl')

In [4]:
import numpy as np
import pandas as pd
index = 7300
df = pd.read_csv('../static/data/news/test.csv')
df.tail()

,data,target
7365,"\nOil Pressure, Oil Temperature\nCoolant Tempe...",7
7366,"\nOh dear, time for me to try to remember my c...",12
7367,\n\n\n-- That means that there cannot be any a...,13
7368,s:\n I have a 1991 Toyota Camry Deluxe for sa...,6
7369,"May 13, 1993 _Five Russian soldiers sentenced...",17


In [ ]:
label = df.target[index] #7300인덱스에 해당하는 타겟
label

In [ ]:
df.data[7000] 
#예를들어 7000번 하면 스트링이 나온다;;(스칼라) #type(df.data[7000])

In [ ]:
# test_data 1차원으로 만드는 방법 1. [리스트]
# 왜냐하면 vectorizer에 input은 스트링의어레이string의 array여야한다.(1차원)리스트로 어레이동급으로 만든다 
# 그렇다고 list('i love you')하면 안됨 ㅋ 'i',' ','a',이렇게만들어짐...ㄴㄴ해
test_data = []
test_data.append(df.data[index]) # 스트링을 통채로 어펜드해야함
test_data

In [ ]:
# test_data 1차원으로 만드는 방법 2. ([넘파이어레이]) (영상31분차)
test_data = df.iloc[index:index+1, :-1].values 
test_data #오.. 2차원이네.. 슬라이싱두번하면 안되겠네

In [ ]:
test_data = df.iloc[index:index+1, 0].values 
test_data 

In [ ]:
#로딩한 모델로
pred = best_tfidf_sv.predict(test_data)

In [ ]:
#결과도 어레이로 나오니까 [0]
pred[0], label

In [ ]:
#화면에보여주려고 만든거라의미없음?ㅋ
news_data.target_names

In [ ]:
target_names = dict(zip(range(20),news_data.target_names))
target_names